## Using custom python function as guard rail validator

### Define function

In [ ]:
from uuid import uuid4


def my_validator(data: str) -> bool:
    return len(data) <= 10


def my_validator_2(data: str) -> bool:
    return len(data) >= 10

### Using guard directly

In [ ]:
from evidently.guardrails import PythonFunction

def process(input: str) -> str:
    PythonFunction(my_validator).validate(input)
    return input

In [ ]:
process("My text")

In [ ]:
from evidently.guardrails import GuardException
# try / catch is to prevent jupyter to fail cell

try:
    process("My tooooo long text")
except GuardException as e:
    import traceback
    traceback.print_exception(e)

### Using guard with decorator

In [ ]:
from evidently.guardrails import guard
from evidently.guardrails import PythonFunction

In [ ]:
@guard(PythonFunction(my_validator), input_arg="data")
def process_with_decorator(data: str) -> str:
    return data

In [ ]:
@guard([PythonFunction(my_validator), PythonFunction(my_validator_2)], input_arg="data")
def process_with_decorator_and_multiple_guards(data: str) -> str:
    return data

In [ ]:
from evidently.guardrails import GuardException
# try / catch is to prevent jupyter to fail cel

val = process_with_decorator_and_multiple_guards("My text")
val

### Handle failed guard in code

In [ ]:
from evidently.guardrails import GuardException
from evidently.guardrails import PythonFunction

def process_with_handle(input: str) -> str:
    try:
        PythonFunction(my_validator).validate(input)
        return input
    except GuardException as e:
        return f"Validation failed: {e}"

In [ ]:
process_with_handle("My tooooo long text")

## Examples of Guards

### PII Check

In [ ]:
from evidently.guardrails import PIICheck


PIICheck().validate("My address is Some city")


In [ ]:
from evidently.guardrails import PIICheck


@guard(PIICheck())
def process_with_pii(input: str) -> str:
    return input

In [ ]:
from evidently.guardrails.guards import ToxicityCheck, NegativityCheck


@guard([ToxicityCheck(), NegativityCheck()])
def process_with_toxicity_and_negativity(input: str) -> str:
    return input

In [ ]:
def process(input: str) -> str:
    data = process_with_pii(input)
    return process_with_toxicity_and_negativity(data)

In [ ]:
messages = [
    "No PII data",
    "My address is Some city",
    "My SSN is 123",
    "This is awful product",
]

for message in messages:
    try:
        process(message)
    except GuardException as e:
        pass

### Includes Words

In [ ]:
from evidently.guardrails import IncludesWords

IncludesWords(["word"], mode="all", lemmatize=True).validate("some words in text")

In [ ]:
@guard(IncludesWords(["word"]))
def process_with_includes(input: str) -> str:
    return input

In [ ]:
@guard(IncludesWords(["word"]))
def process_with_includes(input: str) -> str:
    return input